In [33]:
# !pip install python-binance

import os
from binance.client import Client
import pandas as pd

def get_trading_pairs():
    # Récupérer la liste des paires de trading disponibles sur Binance
    exchange_info = client.get_exchange_info()
    trading_pairs = [symbol['symbol'] for symbol in exchange_info['symbols']]
    nb_of_trading_pairs = len(exchange_info['symbols'])
    return nb_of_trading_pairs, trading_pairs

def get_available_timeframe_on_this_plateform():
    return "ok"

def fetch_ohlcv(symbol, interval='1h', limit=10000):
    # Récupérer les données OHLCV pour une paire de trading spécifique
    candles = client.get_klines(symbol=symbol, interval=interval, limit=limit)
    df = pd.DataFrame(candles, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df

def find_imbalances_after_rise(df):
    imbalances_after_rise = []
    for i in range(1, len(df)-1):
        if df['high'][i-1] < df['low'][i+1]:
            prev_candle = df.iloc[i-1]
            current_candle = df.iloc[i]
            next_candle = df.iloc[i+1]
            price_difference = next_candle['low'] - prev_candle['high']
            imbalance = (*prev_candle, *current_candle, *next_candle, price_difference, current_candle['timestamp'])
            imbalances_after_rise.append(imbalance)
    return imbalances_after_rise

def find_imbalance_after_fall(df):
    imbalances_after_fall = []
    for i in range(1, len(df)-1):
        if df['low'][i - 1] > df['high'][i + 1]:
            prev_candle = df.iloc[i-1]
            current_candle = df.iloc[i]
            next_candle = df.iloc[i+1]
            price_difference = float(prev_candle['low']) - float(next_candle['high'])
            imbalance = (current_candle.name, price_difference, *prev_candle, *current_candle, *next_candle )
            imbalances_after_fall.append(imbalance)
    return imbalances_after_fall

# Récupérer les clés API à partir des variables d'environnement
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

# Initialiser le client Binance
client = Client(api_key, api_secret)

# Récupérer l'historique des prix pour une paire donnée depuis son listing sur la plateforme
candles = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1DAY, "1 Jan, 2015")
candles_count = len(candles)
df = pd.DataFrame(candles, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)
df


,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
timestamp,,,,,,,,,,,
2017-08-17,4261.48000000,4485.39000000,4200.74000000,4285.08000000,795.15037700,1503014399999,3454770.05073206,3427,616.24854100,2678216.40060401,0
2017-08-18,4285.08000000,4371.52000000,3938.77000000,4108.37000000,1199.88826400,1503100799999,5086958.30617151,5233,972.86871000,4129123.31651808,0
2017-08-19,4108.37000000,4184.69000000,3850.00000000,4139.98000000,381.30976300,1503187199999,1549483.73542151,2153,274.33604200,1118001.87008735,0
2017-08-20,4120.98000000,4211.08000000,4032.62000000,4086.29000000,467.08302200,1503273599999,1930364.39032646,2321,376.79594700,1557401.33373730,0
2017-08-21,4069.13000000,4119.62000000,3911.79000000,4016.00000000,691.74306000,1503359999999,2797231.71402728,3972,557.35610700,2255662.55315837,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-03-17,65300.64000000,68904.40000000,64533.00000000,68393.48000000,49742.21589000,1710719999999,3322963634.24251290,2449156,25328.29435000,1692079618.21131060,0
2024-03-18,68393.47000000,68956.00000000,66565.20000000,67609.99000000,55691.08088000,1710806399999,3768277912.41891810,2464515,28529.67219000,1930726566.24924600,0
2024-03-19,67610.00000000,68124.11000000,61555.00000000,61937.40000000,101005.32487000,1710892799999,6484805948.53539280,3593832,48205.28434000,3095258881.70227350,0


In [34]:
imbalances_after_fall = find_imbalance_after_fall(df)

# retourne une liste des jours où il y a eu un imbalance suite à une baisse rapide
list_of_imbalance_start = []
for imbalance in imbalances_after_fall :
    list_of_imbalance_start.append(imbalance[0])
    
list_of_imbalance_start
    

[Timestamp('2017-08-18 00:00:00'),
 Timestamp('2017-09-08 00:00:00'),
 Timestamp('2017-09-13 00:00:00'),
 Timestamp('2017-09-21 00:00:00'),
 Timestamp('2017-11-06 00:00:00'),
 Timestamp('2017-11-10 00:00:00'),
 Timestamp('2017-11-28 00:00:00'),
 Timestamp('2017-11-29 00:00:00'),
 Timestamp('2017-11-30 00:00:00'),
 Timestamp('2017-12-01 00:00:00'),
 Timestamp('2017-12-02 00:00:00'),
 Timestamp('2018-01-08 00:00:00'),
 Timestamp('2018-01-16 00:00:00'),
 Timestamp('2018-01-17 00:00:00'),
 Timestamp('2018-01-18 00:00:00'),
 Timestamp('2018-01-23 00:00:00'),
 Timestamp('2018-01-24 00:00:00'),
 Timestamp('2018-01-29 00:00:00'),
 Timestamp('2018-01-30 00:00:00'),
 Timestamp('2018-01-31 00:00:00'),
 Timestamp('2018-02-01 00:00:00'),
 Timestamp('2018-02-05 00:00:00'),
 Timestamp('2018-02-14 00:00:00'),
 Timestamp('2018-02-15 00:00:00'),
 Timestamp('2018-02-16 00:00:00'),
 Timestamp('2018-02-17 00:00:00'),
 Timestamp('2018-02-21 00:00:00'),
 Timestamp('2018-02-23 00:00:00'),
 Timestamp('2018-02-